In [2]:
import sys, os
if os.path.abspath(os.pardir) not in sys.path:
    sys.path.insert(1, os.path.abspath(os.pardir))
import CONFIG

%reload_ext autoreload
%autoreload 2

In [11]:
import numpy as np
import pandas as pd

from glob import glob
from pathlib import Path
from PIL import Image
import cv2

import matplotlib.pyplot as plt 
import matplotlib.patches as patches 
import seaborn as sns

from bokeh.plotting import figure
from bokeh.io import output_notebook, output_file
from bokeh.models import ColumnDataSource, HoverTool, Panel

import albumentations as albu

In [4]:
INPUT_DIR = CONFIG.CFG.DATA.BASE

/content/data/train


In [8]:
train_imgs = glob(os.path.join(INPUT_DIR, "train", "*"))
test_imgs = glob(os.path.join(INPUT_DIR, "test", "*"))

len(train_imgs), len(test_imgs)

(3422, 10)

In [6]:
train_df = pd.read_csv(os.path.join(INPUT_DIR, "train.csv"))
train_df.head()

,image_id,width,height,bbox,source
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1


In [17]:
all_train_images = pd.DataFrame([Path(path).stem for path in train_imgs], columns=['image_id'])

all_train_images = all_train_images.merge(train_df, on='image_id', how='left')

all_train_images['bbox'] = all_train_images.bbox.fillna('[0, 0, 0, 0]')

# split bbox column
bbox_items = all_train_images.bbox.str.split(',', expand=True)
all_train_images['bbox_xmin'] = bbox_items[0].str.strip('[ ').astype(float)
all_train_images['bbox_ymin'] = bbox_items[1].str.strip(' ').astype(float)
all_train_images['bbox_width'] = bbox_items[2].str.strip(' ').astype(float)
all_train_images['bbox_height'] = bbox_items[3].str.strip(' ]').astype(float)
all_train_images.head()

,image_id,width,height,bbox,source,bbox_xmin,bbox_ymin,bbox_width,bbox_height
0,9e15ceea4,1024.0,1024.0,"[44, 256, 65, 73]",arvalis_1,44.0,256.0,65.0,73.0
1,9e15ceea4,1024.0,1024.0,"[40, 92, 60, 109]",arvalis_1,40.0,92.0,60.0,109.0
2,9e15ceea4,1024.0,1024.0,"[97, 77, 81, 121]",arvalis_1,97.0,77.0,81.0,121.0
3,9e15ceea4,1024.0,1024.0,"[33, 22, 53, 64]",arvalis_1,33.0,22.0,53.0,64.0
4,9e15ceea4,1024.0,1024.0,"[98, 8, 62, 81]",arvalis_1,98.0,8.0,62.0,81.0


In [27]:
# images without head
len(all_train_images) - len(train_df)

49

In [28]:
def get_all_boxes(df, image_id):
    image_bboxes = df[df.image_id == image_id]

    bboxes = []
    for _, row in image_bboxes.iterrows():
        bboxes.append((row.bbox_xmin, row.bbox_ymin, row.bbox_width, row.bbox_height))

    return bboxes

In [ ]:
def plot_image_examples(df, rows=3, cols=3, title="Image examples"):
    fig, axs = plt.subplots(rows, cols, figsize=(10, 10))
    for row in range(rows):
        for col in range(cols):
            idx = np.random.randint(len(df), size=1)[0]